<a href="https://colab.research.google.com/github/SJinLee/XAI2023/blob/main/x1.explainable2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -r train valid
!unzip drive/MyDrive/xai/car_or_truck.zip

In [ ]:
import os, warnings
import matplotlib.pyplot as plt
from matplotlib import gridspec

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
def set_seed(seed=31415):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed(31415)

In [ ]:
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('image', cmap='magma')
warnings.filterwarnings("ignore")

In [ ]:
ds_train_ = image_dataset_from_directory(
    'train',
    labels='inferred',
    label_mode='binary',
    image_size=[128, 128],
    interpolation='nearest',
    batch_size=64,
    shuffle=True,
)
ds_valid_ = image_dataset_from_directory(
    'valid',
    labels='inferred',
    label_mode='binary',
    image_size=[128, 128],
    interpolation='nearest',
    batch_size=64,
    shuffle=False,
)

Found 5117 files belonging to 2 classes.
Found 5051 files belonging to 2 classes.


In [ ]:
# Data Pipeline
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = (
    ds_train_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)
ds_valid = (
    ds_valid_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

In [ ]:
!rm -rf cv-course-models
!unzip drive/MyDrive/xai/base_models.zip

In [ ]:
pretrained_base = tf.keras.models.load_model(
    'cv-course-models/vgg16-pretrained-base',
)
pretrained_base.trainable = False

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    pretrained_base,
    layers.Flatten(),
    layers.Dense(6, activation='relu'),
    layers.Dense(1, activation='sigmoid'),
])

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=20,
    verbose=1,
)

Epoch 1/20
80/80 [==============================] - 20s 175ms/step - loss: 0.5798 - binary_accuracy: 0.7080 - val_loss: 0.5239 - val_binary_accuracy: 0.8214
Epoch 2/20
80/80 [==============================] - 12s 151ms/step - loss: 0.4959 - binary_accuracy: 0.8054 - val_loss: 0.4802 - val_binary_accuracy: 0.8351
Epoch 3/20
80/80 [==============================] - 12s 151ms/step - loss: 0.4560 - binary_accuracy: 0.8358 - val_loss: 0.4576 - val_binary_accuracy: 0.8404
Epoch 4/20
80/80 [==============================] - 16s 208ms/step - loss: 0.4246 - binary_accuracy: 0.8636 - val_loss: 0.4407 - val_binary_accuracy: 0.8472
Epoch 5/20
80/80 [==============================] - 12s 154ms/step - loss: 0.3983 - binary_accuracy: 0.8806 - val_loss: 0.4312 - val_binary_accuracy: 0.8412
Epoch 6/20
80/80 [==============================] - 12s 155ms/step - loss: 0.3756 - binary_accuracy: 0.8933 - val_loss: 0.4203 - val_binary_accuracy: 0.8480
Epoch 7/20
80/80 [==============================] - 12s 15

In [ ]:
import pandas as pd
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot();

In [ ]:
from keras.utils import load_img
# load the image with the required shape
img = load_img('/content/valid/Car/05138.jpeg', target_size=(224, 224))

In [ ]:
from keras.utils import img_to_array
from tensorflow import expand_dims
# convert the image to an array
img = img_to_array(img)
# expand dimensions so that it represents a single 'sample'
img = expand_dims(img, axis=0)
from keras.applications.vgg16 import preprocess_input
# prepare the image (e.g. scale pixel values for the vgg)
img = preprocess_input(img)

In [ ]:
img_plot = img[0,:,:,::-1].numpy()
plt.imshow((img_plot-img_plot.min())/(img_plot.max()-img_plot.min()))

In [ ]:
img.numpy().max()

In [ ]:
img.numpy().min()

In [ ]:
for a in ds_train:
    break

In [ ]:
a[0][0].shape

In [ ]:
a[0][0].numpy().max()

In [ ]:
a[0][0].numpy().min()

In [ ]:
from keras.utils import load_img
# load the image with the required shape
img = load_img('/content/valid/Car/05138.jpeg', target_size=(128, 128))
from keras.utils import img_to_array
from tensorflow import expand_dims
# convert the image to an array
img = img_to_array(img)
# expand dimensions so that it represents a single 'sample'
img = expand_dims(img, axis=0)
img = img / 255

In [ ]:
img.numpy().max(),img.numpy().min()

In [ ]:
img_plot = img[0,:,:,:].numpy()
plt.imshow(img_plot)

In [ ]:
model.predict(img)

In [ ]:
img_numpy = img.numpy()
img_numpy[:,80:100,10:30,:] = 0

In [ ]:
plt.imshow(img_numpy[0,:,:,:])

In [ ]:
model.predict(img_numpy)

In [ ]:
from keras.utils import load_img
# load the image with the required shape
img = load_img('/content/valid/Car/05138.jpeg', target_size=(128, 128))
from keras.utils import img_to_array
from tensorflow import expand_dims
# convert the image to an array
img = img_to_array(img)
# expand dimensions so that it represents a single 'sample'
img = expand_dims(img, axis=0)
img = img / 255

img_numpy = img.numpy()
img_numpy[:,70:110,10:120,:] = 0

plt.imshow(img_numpy[0,:,:,:])

model.predict(img_numpy)

In [ ]:
from keras.utils import load_img
# load the image with the required shape
img = load_img('/content/valid/Truck/05157.jpeg', target_size=(128, 128))
from keras.utils import img_to_array
from tensorflow import expand_dims
# convert the image to an array
img = img_to_array(img)
# expand dimensions so that it represents a single 'sample'
img = expand_dims(img, axis=0)
img = img / 255

img_numpy = img.numpy()
# img_numpy[:,70:110,10:120,:] = 0

plt.imshow(img_numpy[0,:,:,:])

model.predict(img_numpy)